<a href="https://colab.research.google.com/github/Junseokee/Deep_Learning/blob/main/220919_%EB%AC%B8%EC%84%9C%EC%97%90_%EA%BC%AC%EB%A6%AC%ED%91%9C_%EB%8B%AC%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ratsnlp
# 교재 실습을 위한 의존성 패키지

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.0 MB/s 
     |████████████████████████████████| 582 kB 15.2 MB/s 
     |████████████████████████████████| 2.8 MB 56.5 MB/s 
     |████████████████████████████████| 57 kB 7.1 MB/s 
     |████████████████████████████████| 419 kB 86.0 MB/s 
     |████████████████████████████████| 3.3 MB 50.8 MB/s 
     |████████████████████████████████| 120 kB 80.9 MB/s 
     |████████████████████████████████| 880 kB 80.7 MB/s 
     |████████████████████████████████| 96 kB 6.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=a7f6d1ec03cc3d01d2ed95b79757a908087616bcb358c5a94e16c7ed0e9ad5c7
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstallin

### 1단계 코랩 노트북 초기화하기

In [ ]:
# 구글드라이브와 연결
from google.colab  import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 2단계 환경 설정하기

In [ ]:
# 모델 환경 설정
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments

args = ClassificationTrainArguments(
    pretrained_model_name='beomi/kcbert-base',
    downstream_corpus_name='nsmc', # nsmc밖에없음
    downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-doccls',
    batch_size = 32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=128,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

- pretrained_model_name : 프리트레인 마친 언어모델의 이름(단 해당모델은 허깅페이스 라이브러리에 등록되어있어야함)
- downstream_corpus_name : 다운스트림 데이터의 이름
- downstream_model_dir : 파인튜닝된 모델의 체크포인트가 저장될 위치
- batch_size : 배치크기. 하드웨어 가속기로 GPU를 선택했다면 32, TPU를 선택했다면 4개 코랩환경에 TPU는 보통 8개의 코어가 할당되는데 batch_size는 코어별로 적용되는 배치크기이기 때문에 이렇게 설정
- max_seq_length :  토큰 기준 입력 문장 최대길이.
- tpu_cores

In [ ]:
# 랜덤시드고정
from  ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


In [ ]:
# 로거 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-doccls', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=4, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-doccls', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=

### 3단계 말뭉치 내려 받기

In [ ]:
args

ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-doccls', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=4, fp16=False, tpu_cores=0)

In [ ]:
# 말뭉치 다운로드
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=True,
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 155MB/s]
[nsmc] download ratings_test.txt: 4.90MB [00:00, 58.0MB/s]


### 4단계 토크나이저 준비하기
  - 데이터의 기본 단위는 텍스트 형태의 문장이다. 토큰화란 ㅁ누장을 토큰 시퀀스로 분절하는 과정이다. 이장 튜토리얼에서 사용하는 모델은 자연어 문장을 분절한 토큰 시퀀스를 입력으로 받는다.
  - 다음코드를 실행해 이준범님이 공개하신 kcbert-base모델이 사용하는 토크나이저를 선언



In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False, # 소문자 변환여부 한글이라 False
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

### 5단계 데이터 전처리하기
  - 딥러닝 모델을 학습하려면 학습데이터를 배치 단위로 지속적으로 모델에 공급해야한다.
  - 파이토치에서는 이역할을 데이터 로더가 수행
    - 데이터 로더는 데이터셋에서 랜덤 인스턴스를 배치로 묶어서 학습 모델에 전달
   
- 4단계 에서는 NsmcCorpus는 CSV 파일 형식의 NSMC데이터를 문장(영화리뷰) + 레이블(긍정, 부정) 형태로 읽어들이는 역할
- ClassificationDataset는 그림 4-4의 데이터셋 역할을 수행

In [ ]:
# 학습데이터셋 구축
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset
corpus = NsmcCorpus()
# /usr/local/lib/python3.7/dist-packages/ratsnlp/nlpbook/classification/corpus.py
train_dataset = ClassificationDataset(
    args = args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode='train',
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 32.490 s]
INFO:ratsnlp:tokenize sentences [took 32.490 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:sentence: 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:tokens: [CLS] 아 더 ##빙 . . 진짜 짜증나네 ##요 목소리 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

- ClassificationDataset은 입력받은 자연어를 input_ids, attention_mask, otoken_type_ids과 같은 정보로 변환
  - input_ids : 인덱스로 변환된 토큰
  - attention_mask: 해당 토큰이 패딩 토큰인지(0) 아닌지(1) 나타낸다.

In [ ]:
train_dataset[0]

ClassificationFeatures(input_ids=[2, 2170, 832, 5045, 17, 17, 7992, 29734, 4040, 10720, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
# 학습 데이터 로더 구축
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

- sampler: 샘플링 방식을 정의
- collate_fn : 인스턴스를 배치로 만드는 역할을 하는 함수 텐서 형태로 바꾸는 등의 역할을 수행

In [ ]:
# 평가용 데이터 로더 구축
from torch.utils.data import SequentialSampler

val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode='test',
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_test_BertTokenizer_128_nsmc_document-classification [took 3.359 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_test_BertTokenizer_128_nsmc_document-classification [took 3.359 s]


### 6단계 모델 불러오기


In [ ]:
# 모델 초기화
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

### 7단계 모델 학습시키기
- 파이토치 라이트닝이 제공하는 라이트닝 모듈(LightningModule) 클래스를 상속받아 태스크를 정의

In [ ]:
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)

In [ ]:
trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/optimizer.py:381: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  f"Found unsupported keys in the optimizer configuration: {set(extra_keys)}", category=RuntimeWarning
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.680   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


## 실습?

### 1단계 코랩 노트북 초기화하기


In [ ]:
!mkdir /content/drive/MytDrive/nlpbook
!mkdir /content/drive/MytDrive/nlpbook/checkpoint-doccls

mkdir: cannot create directory ‘/content/drive/MytDrive/nlpbook’: No such file or directory
mkdir: cannot create directory ‘/content/drive/MytDrive/nlpbook/checkpoint-doccls’: No such file or directory


In [ ]:
!pip install -U --no-cache-dir gdown --pre
print("Downloading...")
!gdown --id {"1-6gU3bZbJoaQd9CFk_vHy9JcFWodh6yK"} -O /content/drive/MyDrive/nlpbook/checkpoint-doccls/epoch=0-val_loss=0.45.ckpt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.5.1-py3-none-any.whl size=14951 sha256=9676817bc2c31c76345a98ada05c0b5852b8fe5a7c10e6be1fb644252b2732a9
  Stored in directory: /tmp/pip-ephem-wheel-cache-sldgc2oa/wheels/3d/ec/b0/a96d1d126183f98570a785e6bf8789fca559853a9260e928e1
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
Downloading...
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-6gU3bZbJoaQd9CFk_vHy9JcFWodh6yK
To:

### 2단계 환경 설정하기

In [ ]:
# 모델 환경 설정
from ratsnlp.nlpbook.classification import ClassificationDeployArguments
args = ClassificationDeployArguments(
    pretrained_model_name='beomi/kcbert-base',
    downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-doccls',
    max_seq_length=128,
)

downstream_model_checkpoint_fpath: /content/drive/MyDrive/nlpbook/checkpoint-doccls/epoch=0-val_loss=0.45.ckpt


In [ ]:
args

ClassificationDeployArguments()

### 3단계 토크나이저 및 모델 불러오기

In [ ]:
# 토크나이저 생략

In [ ]:
# 체크포인트 로드
import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device('cpu')
)

In [ ]:
# BERT 설정 로드
from transformers import BertConfig
pretrained_model_config =BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape[0],
)

In [ ]:
# BERT 모델 초기화
from transformers import BertForSequenceClassification
model = BertForSequenceClassification(pretrained_model_config)

In [ ]:
# 체크포인트 읽기
model.load_state_dict({k.replace('model.',""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

<All keys matched successfully>

In [ ]:
# 평가 모드 전환
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# INFERENCE
def inference_fn(sentence):
  inputs = tokenizer(
      [sentence],
      max_length=args.max_seq_length,
      padding='max_length',
      truncation=True,
  )
  with torch.no_grad():
    outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
    prob = outputs.logits.softmax(dim=1)
    positive_prob = round(prob[0][1].item(),4)
    negative_prob = round(prob[0][0].item(),4)
    pred = '긍정 (positrive)' if torch.argmax(prob) == 1 else '부정 (negative)'
  return {
      'sentence' : sentence,
      'prediction' : pred,
      'positive_data' : f'긍정 {positive_prob}',
      'negative_data' : f'부정 {negative_prob}',
      'positive_width' : f'{positive_prob *100}%',
      'negative_width' : f'{negative_prob*100}%',
  }

In [ ]:
inference_fn('이게 영화냐 꿈이지')

{'sentence': '이게 영화냐 꿈이지',
 'prediction': '부정 (negative)',
 'positive_data': '긍정 0.0658',
 'negative_data': '부정 0.9342',
 'positive_width': '6.58%',
 'negative_width': '93.42%'}

### 5단계 웹 서비스 시작하기
ngrok은 코랩 로컬에서 실행중인웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구입니다.
ngrok을 실행하려면 회원가입 후 로그인을 한뒤 이곳에 접속해

In [ ]:
!mkdir /root/.ngrok2 && echo "authtoken: 2Ey1spuRdQ0sCjMXTEvvtR08odp_5avYiwqBsCqUmafHMREBt" > /root/.ngrok2/ngrok.yml


In [ ]:
from ratsnlp.nlpbook.classification import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.classification.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://81fb-104-199-127-55.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


- https://github.com/SKTBrain/KoBERT